## 라이브러리 import 및 설정

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import gc
from tensorflow.keras.backend import clear_session

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords

from pathlib import Path

import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam

import warnings 

In [3]:
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 학습데이터 로드

In [4]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 2020 

In [5]:
algo_name = 'mlp'
feature_name = 'tfidf-pca' #학습할 피처- tfidf 피처를 pca한 데이터
feature_target = 'feature_target'

model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv' #학습할 피처 가져오기
feature_target_file = feature_dir / f'{feature_target}.csv' #target 데이터 가져오기

p_val_ver7_file = val_dir / f'{model_name}_oof_pred_ver7.csv'
p_tst_ver7_file = tst_dir / f'{model_name}_test_pred_ver7.csv'
sub_ver7_file = sub_dir / f'{model_name}_ver7.csv'

## 데이터 셋 불러오기

In [6]:
X = pd.read_csv(feature_file, index_col=0).values
y = pd.read_csv(feature_target_file,index_col=0).values

X_7 = X[:54879]
X_tst_7 = X[54879:]


In [7]:
print(X_7.shape)
print(X_tst_7.shape)
print(type(X_7))

(54879, 500)
(19617, 500)
<class 'numpy.ndarray'>


## mlp 모델 학습

In [8]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [9]:
def get_model(number):
    inputs = Input(shape=(number,))
    x = Dense(128, activation='relu')(inputs)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

In [10]:
p_val_ver7 = np.zeros((X_7.shape[0], n_class))
p_tst_ver7 = np.zeros((X_tst_7.shape[0], n_class))

for X, test in [(X_7, X_tst_7)]:  
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es])
       
        # Predict
        if X.shape[1]==500:
            p_val_ver7[i_val, :] = clf.predict(X[i_val])
            p_tst_ver7 += clf.predict(test) / n_class
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr Ver7 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver7, axis=1)) * 100:8.4f}%')
print(f'lr Ver7 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver7):8.4f}')


Training model for CV #1
Epoch 1/10
86/86 [==============================] - 1s 7ms/step - loss: 1.4803 - val_loss: 1.2990
Epoch 2/10
86/86 [==============================] - 0s 4ms/step - loss: 1.1005 - val_loss: 0.9587
Epoch 3/10
86/86 [==============================] - 0s 4ms/step - loss: 0.8697 - val_loss: 0.8391
Epoch 4/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7923 - val_loss: 0.7999
Epoch 5/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7626 - val_loss: 0.7845
Epoch 6/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7479 - val_loss: 0.7774
Epoch 7/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7392 - val_loss: 0.7722
Epoch 8/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7332 - val_loss: 0.7698
Epoch 9/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7288 - val_loss: 0.7659
Epoch 10/10
86/86 [==============================] - 0s 4ms/step - loss: 0.7248 - val_loss: 0

Exception: Data must be 1-dimensional

## 제출 파일 생성 및 기타 파일 생성

In [ ]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver1
sub[sub.columns] = p_tst_ver7
sub.to_csv(sub_ver7_file)


In [ ]:
# p_val 파일 생성 -> oof

# Ver1
np.savetxt(p_val_ver7_file, p_val_ver7, fmt='%.18f', delimiter=',')



In [ ]:
# p_tst 파일 생성 -> test 

# Ver1
np.savetxt(p_tst_ver7_file, p_tst_ver7, fmt='%.18f', delimiter=',')

